# 3D Brain Visualizations

## 3D topographic brain map

The 3D topographic brain map provides a view of voltage measurements as a heatmap converted to estimated position on the brain. There are 2 plot options with different backends: 

1) [matplotlib](#matplotlib) (the version found on the UI)

2) [pyVista](#pyvista) (a better rendering but incompatible with the UI)

Both plots can be generated as an animation to view changes over time or as a standalone plot.

![](instruction_imgs/presentattion_brain.gif)

## General Setup
### Import required modules

In [1]:
from simpl_eeg import topomap_3d_brain, eeg_objects

In [2]:
import warnings
warnings.filterwarnings('ignore')

```{note}
Please include the line below in your IDE so that the changes would be simultaneously reflected when you make a change to the python scripts.
```

In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

### Create epoched data
For additional options see [Creating EEG Objects](eeg_objects.html#intro) section.

In [5]:
experiment_folder = "../../data/109"
epochs = eeg_objects.Epochs(experiment_folder)

frame_steps = 100
epoch = epochs.skip_n_steps(frame_steps)

Reading /Users/mpin/Documents/MDS/capstone/simpl_eeg_capstone/data/109/fixica.fdt
Not setting metadata
Not setting metadata
33 matching events found
Setting baseline interval to [-0.2998046875, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 33 events and 2049 original time points ...
0 bad epochs dropped


### Generate forward and inverse (optional)

```{note}
- Before an animation or plot can be generated, a **"forward"** and **"inverse"** (abbreviated as **"stc"**) must first be generated. If they are not provided to either of the plotting animations they will be automatically generated, **HOWEVER** this will increase the time it takes to generate the figure.

- The forward/inverse are used to retrieve a brain model to attach the EEG data and to do some of the mapping calculations. The forward downloads 'fsaverage' MRI data which represents a brain averaged out from dozens of different patients.
```

#### Generate Forward

In [6]:
fwd = topomap_3d_brain.create_fsaverage_forward(epoch)

0 files missing from root.txt in /Users/mpin/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /Users/mpin/mne_data/MNE-fsaverage-data/fsaverage
Source space          : /Users/mpin/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif
MRI -> head transform : /Users/mpin/opt/miniconda3/envs/simpl_eeg/lib/python3.9/site-packages/mne/data/fsaverage/fsaverage-trans.fif
Measurement data      : instance of Info
Conductor model   : /Users/mpin/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Reading /Users/mpin/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif...
Read 2 source spaces a total of 20484 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999994  0.003552  0.000202      -1.76 mm
    -0.003558  0.998389  0.056626      31.09 mm
    -0.000001 -0.056626  0.998395      39.60 mm
     0.000000  0.00

#### Generate Inverse

In [7]:
stc = topomap_3d_brain.create_inverse_solution(epoch, fwd)

Computing rank from data with rank=None
    Using tolerance 1.4e-12 (2.2e-16 eps * 19 dim * 3.4e+02  max singular value)
    Estimated rank (eeg): 19
    EEG: rank 19 computed from 19 data channels with 0 projectors
Reducing data rank from 19 -> 19
Estimating covariance using EMPIRICAL
Done.
Estimating covariance using SHRUNK
Done.
Using cross-validation to select the best estimator.
Number of samples used : 21
log-likelihood on unseen data (descending order):
   shrunk: -65.605
   empirical: -257.367
selecting best estimator: shrunk
[done]
Converting forward solution to surface orientation
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Computing inverse operator with 19 channels.
    19 out of 19 channels remain after picking
Selected 19 channels
Creating the depth weighting matrix...
    19 EEG channels
    limit = 20485/20484 = 9.583

<a id="matplotlib"></a>
## Create a matplotlib 3D brain animation

### Simple Plot (+ auto-generating forward and inverse)

#### Define parameters (simple)

A detailed description of all parameters can be found in the `topomap_3d_brain.animate_matplot_brain` docstring:

In [8]:
help(topomap_3d_brain.animate_matplot_brain)

Help on function animate_matplot_brain in module simpl_eeg.topomap_3d_brain:

animate_matplot_brain(epoch, stc='auto', views=['lat', 'dor', 'fro'], size=200, hemi='both', colormap='mne', colorbar=True, colormap_limit_type='lims', cmin=None, cmid=None, cmax=None, spacing='oct5', smoothing_steps=2, timestamp=True, frame_rate=12, **kwargs)
    Creates an animated view of all timestamp observations an mne.epochs.Epochs data using a matplotlib backend.
    If multiple views are used then speed becomes significantly slower. Colorbar placement may be inconsistent.
    
    Parameters:
        epoch: mne.epochs.Epochs or mne.evoked.EvokedArray
            MNE epochs or evoked object containing portions of raw EEG data built around specified
            timestamp(s) The inverse solution will be built based on the data in the specified epoch.
    
        stc: mne.source_estimate.SourceEstimate | 'auto'
            'inverse_solution' to generate the plot from. If set to "auto" (default) then an 

#### Define parameters

In [9]:
# change values below to values of interest

# arguments built into the package
views=['lat', 'dor']
size=200
hemi='both'
colormap='mne'
colorbar=True
colormap_limit_type='lims'
cmin=-2
cmax=2
spacing='oct5'
smoothing_steps=5
timestamp=True
frame_rate=12


# some useful (and tested) arguments from the
# MNE.viz.plot_topomap function (see **kwargs) include...
surface='inflated'
cortex='classic'

#### Generate animation with matplotlib backend

In [10]:
%%capture

matplotlib_animation = topomap_3d_brain.animate_matplot_brain(
    epoch,
    stc=stc,
    views=views,
    hemi=hemi,
    colormap=colormap,
    colorbar=colorbar,
    colormap_limit_type=colormap_limit_type,
    cmin=cmin,
    cmax=cmax,
    spacing=spacing,
    smoothing_steps=smoothing_steps,
    timestamp=timestamp,
    frame_rate=frame_rate,
    surface=surface,
    cortex=cortex
)

from IPython.display import HTML
video = HTML(matplotlib_animation.to_jshtml())

In [11]:
video

### Saving the animation

#### Save as gif

```python
anim_brain = topomap_3d_brain.animate_matplot_brain(epoch, stc = stc, views = 'lat', hemi = 'lh')

gif_file_path = "examples/topomap_3d_brain.gif" 
anim_brain.save(gif_file_path, fps=5, dpi=300)
```

#### Save as mp4

```python
mp4_file_path = "examples/topo_2d.mp4"
anim_brain.save(mp4_file_path, fps=5, dpi=300)
```

```{note}
If `FFMpegWriter` does not work on your computer you can save the file as a gif first and then convert it into mp4 file by running the code below.
```
```python
import moviepy.editor as mp

clip = mp.VideoFileClip(gif_file_path)
clip.write_videofile(mp4_file_path)
```

## Create a matplotlib 3D brain figure

### Generating a matplotlib plot

### Define parameters
A detailed description of all animation parameters can be found in the `topomap_3d_brain.plot_topomap_3d_brain` docstring:

In [12]:
help(topomap_3d_brain.plot_topomap_3d_brain)

Help on function plot_topomap_3d_brain in module simpl_eeg.topomap_3d_brain:

plot_topomap_3d_brain(epoch, stc='auto', recording_number=0, backend='matplotlib', views=['lat', 'fro', 'dor'], view_layout='horizontal', size=300, hemi='both', colormap='mne', colormap_limit_type='lims', cmin=None, cmid=None, cmax=None, colorbar=True, time_viewer='auto', background='black', foreground='white', spacing='oct6', smoothing_steps=3, figure=None, **kwargs)
    Creates a still image of the epochs or stc data mapped to the brain using the mne.SourceEstimate.plot
    function. Written to work with either a 'pyvista' or 'matplotlib' backend. Pyvista will be faster but
    at the expense of the returned object being less compatible with other functions. The 'matplotlib' backend
    will return a matplotlib figure which is widely used in python.
    
    Parameters:
        epoch: mne.epochs.Epoch
            MNE epochs object containing portions of raw EEG data built around specified
            timest

In [13]:
%%capture
matplot_brain_fig = topomap_3d_brain.plot_topomap_3d_brain(epoch, stc=stc, recording_number = 2)

### Save the plot
You can change the file to different formats by changing the format argument in the function. It supports `png`, `pdf`, `svg`.
```python
file_path = "examples/topomap_3d_brain.svg"  
matplot_brain_fig.savefig(file_path, format='svg')
```

<a id="pyvista"></a>
## Create a pyVista 3D brain animation

```{note}
The dependencies for the following functions are not included with the base instalation of the simpl_eeg package due to compatability issues with certain operating systems. If you wish to use them please install 'PyVista', 'PyQt5', 'pyvistaqt', and 'pyqt5-qt5' through your preferred package installation method.
```

### Generate figure with pyvista backend

```python
pyvista_brain_fig = topomap_3d_brain.plot_topomap_3d_brain(epoch, stc = stc, backend = 'pyvista')
```

### Save animation with pyvista backend

```python
topomap_3d_brain.save_animated_topomap_3d_brain(pyvista_brain_fig, filename = "brain_animation.gif")
```